In [1]:
from PIL import Image, ImageOps
from chainer import using_config
from chainer.cuda import to_cpu
from chainer.serializers import load_npz
from errno import EEXIST
from glob import glob
from os import makedirs
from os.path import split
from tqdm import tnrange
from utils import Model

In [2]:
device = 0
fp = sorted(glob('../data_set/validation_input/*.jpg'))

In [3]:
model = Model()

load_npz('../model.npz', model)

if device >= 0:
    model.to_gpu(device)

In [4]:
try:
    makedirs('../data_set/validation_output')
except OSError as exception:
    if exception.errno != EEXIST:
        raise

for i in tnrange(len(fp)):
    x = model.xp.asarray(Image.open(fp[i]).convert('RGB').resize((138, 200), Image.LANCZOS), 'f').transpose(2, 0, 1)[None]

    with using_config('train', False):
        y_hat = Image.fromarray(to_cpu(model(x).data[0, 0]).astype('uint8')).resize((275, 400), Image.LANCZOS)

    y_hat.save('../data_set/validation_output/{}'.format(split(fp[i])[1])) # *contents* of the validation_output directory
                                                                           # should be zipped and submitted

HBox(children=(IntProgress(value=0, max=8400), HTML(value='')))